# Question 1: Redpanda version


Now let's find out the version of redpandas.

For that, check the output of the command rpk help inside the container. The name of the container is redpanda-1.

Find out what you need to execute based on the help output.

What's the version, based on the output of the command you executed? (copy the entire version)

rpk version v23.3.6 (rev 78642d6969)
redpanda@bf0f76ef0fd5:/$ rpk version
Version:     v23.3.6
Git ref:     78642d6969
Build date:  2024-02-21T20:04:31Z
OS/Arch:     linux/amd64
Go version:  go1.21.3

Redpanda Cluster
  node-0  v23.3.6 – 78642d69697481ab985a808d95d7943811bfeca9

# Question 2. Creating a topic


In [ ]:
edpanda@bf0f76ef0fd5:/$ rpk topic create test-topic

edpanda@bf0f76ef0fd5:/$ rpk topic create test-topic

TOPIC       STATUS
test-topic  OK

# Question 3. Connecting to the Kafka server


We need to make sure we can connect to the server, so later we can send some data to its topics

First, let's install the kafka connector (up to you if you want to have a separate virtual environment for that)



In [1]:
!pip install kafka-python



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Let's try to connect to our server:



In [1]:
import json
import time 

from kafka import KafkaProducer


In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

Provided that you can connect to the server, what's the output of the last command?



True


# Question 4. Sending data to the stream

In [3]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t2= time.time()
producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')
print(f'took {(t2 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.50 seconds
took 0.50 seconds


## Sending the taxi data


In [131]:
#!pip install pyspark

In [4]:
import pandas as pd

In [5]:
df_green = pd.read_csv('../green_tripdata_2019-10.csv')

/tmp/ipykernel_3630350/16377989.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv('../green_tripdata_2019-10.csv')


In [6]:
df_green.shape

(476386, 20)

In [7]:
df_green = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance',
       'tip_amount']]

In [8]:
df_green.columns

Index(['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID',
       'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount'],
      dtype='object')

In [9]:
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    print(row_dict)
    break

    # TODO implement sending the data here

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}


In [10]:
df_green.shape

(476386, 7)

# Question 5: Sending the Trip Data


Create a topic green-trips and send the data there
How much time in seconds did it take? (You can round it to a whole number)
Make sure you don't include sleeps in your code


rpk topic create green-trips

redpanda@bf0f76ef0fd5:/$ rpk topic create green-tripsps
TOPIC        STATUS
green-trips  OK


In [11]:
t0 = time.time()

topic_name = 'green-trips'
'''
for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
'''
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
    #print(row_dict)
    #break
    
t2= time.time()
producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')
print(f'took {(t2 - t0):.2f} seconds')

took 31.89 seconds
took 31.88 seconds


## Creating the PySpark consumer


In [12]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/04/05 19:29:53 WARN Utils: Your hostname, loula resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
24/04/05 19:29:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hhammad/miniconda3/envs/stenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hhammad/.ivy2/cache
The jars for the packages stored in: /home/hhammad/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a19d8e5e-fc47-4158-b0ad-f27d1e435a08;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 356ms :: artifacts dl 12ms
	

### Now we can connect to the stream:



In [13]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()


In [14]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/05 19:30:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7e4b70ba-98c7-48bf-8cb0-87e46031c122. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 19:30:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/05 19:30:12 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"VendorID": 2.0, "lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "store_and_fwd_flag": "N", "RatecodeID": 1.0, "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "fare_amount": 18.0, "extra": 0.5, "mta_tax": 0.5, "tip_amount": 0.0, "tolls_amount": 0.0, "ehail_fee": NaN, "improvement_surcharge": 0.3, "total_amount": 19.3, "payment_type": 2.0, "trip_type": 1.0, "congestion_surcharge": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 5, 17, 15, 36, 186000), timestampType=0)


# Question 6. Parsing the data


The data is JSON, but currently it's in binary format. We need to parse it and turn it into a streaming dataframe with proper columns.

Similarly to PySpark, we define the schema

In [40]:
df_green.columns

Index(['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID',
       'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount'],
      dtype='object')

In [41]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())


In [42]:
from pyspark.sql import functions as F


In [43]:

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast("string"), schema).alias("data")) \
  .select("data.*")

In [24]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/04/05 19:17:52 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5ea88144-b59c-43cd-84ef-4b8856f86f33. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/05 19:17:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/05 19:17:52 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [28]:
query = green_stream.writeStream.start()

IllegalArgumentException: 'path' is not specified.

In [27]:
query.stop()

In [32]:
from pyspark.sql import functions as F



In [ ]:
green_stream = green_stream \
  .select(F.from_json(F.col('value').cast('string'), schema).alias("data")) \
  .select("data.*")

In [33]:
from pyspark.sql import functions as F
topic_name = 'green-trips'

df = spark.readStream \
     .format("kafka") \
     .option("kafka.bootstrap.servers", "localhost:9092") \
     .option("subscribe", topic_name) \
     .option("startingOffsets", "earliest") \
     .load() \
     .select(F.col('key').cast('string'),
             F.from_json(F.col("value").cast("string"), schema).alias('value'),
             F.col('topic'))

In [34]:
type(df)

pyspark.sql.dataframe.DataFrame

In [36]:
len(df.columns)

3

In [37]:
df.printSchema()


root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- lpep_pickup_datetime: string (nullable = true)
 |    |-- lpep_dropoff_datetime: string (nullable = true)
 |    |-- PULocationID: integer (nullable = true)
 |    |-- DOLocationID: integer (nullable = true)
 |    |-- passenger_count: double (nullable = true)
 |    |-- trip_distance: double (nullable = true)
 |    |-- tip_amount: double (nullable = true)
 |-- topic: string (nullable = true)



# Question 7: Most popular destination


In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

popular_destinations=

In [ ]:
# Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [15]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()


NameError: name 'popular_destinations' is not defined